In [1]:
import os
import textwrap
from pathlib import Path
import shutil

from dotenv import find_dotenv, load_dotenv, dotenv_values

import openai
from langchain.chains import LLMChain
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import YoutubeLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts.chat import (ChatPromptTemplate,SystemMessagePromptTemplate, HumanMessagePromptTemplate)
from pytube import Playlist, YouTube

import cohere

import autogen

In [2]:
config = dotenv_values("../.env")

In [3]:
embeddings = OpenAIEmbeddings(openai_api_key=config['OPENAI_API_KEY'])
co = cohere.Client(config['COHERE_API_KEY'])

In [4]:
db = FAISS.load_local("../data/Huberman_Lab_Episodes_with_Guests", embeddings)

In [5]:
def get_response_from_query(query, k=4, rerank=False):
    """
    gpt-3.5-turbo can handle up to 4097 tokens. Setting the chunksize to 1000 and k to 4 maximizes
    the number of tokens to analyze.
    """

    if rerank:
        k_ = k * 5
    else:
        k_ = k
    docs = db.similarity_search(query, k=k_)
    if rerank:
        list_doc_contents = list(map(lambda x: x.page_content, list(docs)))
        reranked_docs = co.rerank(
        model = 'rerank-english-v2.0',
        query = query,
        documents = list_doc_contents,
        top_n = k,
        )
        indices = list(map(lambda x: x.index, reranked_docs.results))
        reranked_subset = [docs[i] for i in indices]
        docs = reranked_subset
            
    docs_page_content = " ".join([d.page_content for d in docs])

    chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.2)

    # Template to use for the system message prompt
    template = """
        You are a helpful assistant that that can answer questions about youtube videos 
        based on the video's transcript: {docs}
            
        Only use the factual information from the transcript to answer the question.
            
        If you feel like you don't have enough information to answer the question, say "I don't know".
            
        Your answers should be verbose and detailed.
        """

    system_message_prompt = SystemMessagePromptTemplate.from_template(template)

    # Human question prompt
    human_template = "Answer the following question: {question}"
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

    chat_prompt = ChatPromptTemplate.from_messages(
        [system_message_prompt, human_message_prompt]
    )

    chain = LLMChain(llm=chat, prompt=chat_prompt)

    response = chain.run(question=query, docs=docs_page_content)
    response = response.replace("\n", "")
    return response

In [6]:
get_response_from_query("What is the difference between the sympathetic and parasympathetic nervous system?")

'The sympathetic and parasympathetic nervous systems are two branches of the autonomic nervous system, which controls involuntary bodily functions. While they both play important roles in regulating our body\'s responses, they have distinct functions and effects.The sympathetic nervous system is often referred to as the "fight or flight" response system. It is responsible for preparing the body for action in response to perceived threats or stressors. When activated, the sympathetic nervous system increases heart rate, dilates blood vessels, and redirects blood flow to the muscles, preparing the body for physical exertion. It also triggers the release of stress hormones like adrenaline, which heightens alertness and increases energy levels. In essence, the sympathetic nervous system helps us respond to emergencies and mobilize our resources for survival.On the other hand, the parasympathetic nervous system is often referred to as the "rest and digest" response system. Its main function

In [7]:
config_list = [
    {
        'model': 'gpt-4',
        'api_key': config['OPENAI_API_KEY']
    }
]

In [12]:
llm_config = {
    "seed": 42,  # change the seed for different trials
    "temperature": 0,
    "config_list": config_list,
    "request_timeout": 120,
    "functions": [
        {
            "name": "answer_neuroscience_question",
            "description": "Get fact-based responses related to neuroscience and the effectiveness of different supplements and interventions",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The question to ask the model"
                    }
                }
            },
            "required": ["question"],
        }
    ]
}

In [34]:
article_writer_prompt_simple = """
Article writer. The article must be appealing to the wider audience and the information has to be fact-checked. 
In order to check facts, ask the fact checker for the information you need to perform the search. After you have written the article by iterating with fact checker, ask the user if the content is ready to publish or it needs more work.
When you respond with the status add the word TERMINATE
"""

article_writer_prompt_complex = """
You are an article writer. Before diving into the writing process, it's crucial to understand who you're writing for. What interests them? What questions are they seeking answers to? This will not only make your article more appealing but also help you gauge which facts to prioritize.
Once you have a clear idea of your topic and the direction of your article, loop in a fact-checker. They will be your ally in ensuring that your content is not only compelling but also accurate.

Tip: It's essential to maintain open communication with your fact-checker and SEO optimizer. Whenever you come across a piece of information you're unsure about, reach out. They are there to help!
With your audience in mind and a fact-checker by your side, start writing. Remember, the first draft is just that - a draft. Don't be disheartened if it's not perfect. Writing is an iterative process.
After your initial draft, revisit your work. Look for areas that can be enhanced, both in terms of content and accuracy. This is where your fact-checker will shine. Engage with them, ask questions, and make revisions based on their feedback.
Once you're satisfied with your piece, do one final check. This is the time to ensure that your article flows well, is free from errors, and, most importantly, is factually sound.
After you have written the article by iterating with fact checker, ask the user if the content is ready to publish or it needs more work.
When you respond with the status add the word TERMINATE
"""

fact_checker = autogen.AssistantAgent(
    name="fact_checker",
    llm_config=llm_config,
    human_input_mode="NEVER",
    code_execution_config=False,
    system_message="""Fact checker, you help the user find the relevant, fact-based information. 
    Only use the tools provided to do the search. Only execute the search after you have all the information needed. 
    Ask the article writer for the information you need to perform the search, always add the word "BRKT"" at the end of your question.
    When you respond with the status add the word TERMINATE
    """,
    function_map={
        "answer_neuroscience_question": get_response_from_query
    }
)

article_writer = autogen.AssistantAgent(
    name="article_writer",
    llm_config=llm_config,
    human_input_mode="NEVER",
    code_execution_config=False,
    system_message=article_writer_prompt_complex,
    function_map={
        "answer_neuroscience_question": get_response_from_query
    }
)

seo_optimizer = autogen.AssistantAgent(
    name="seo_optimizer",
    llm_config=llm_config,
    human_input_mode="NEVER",
    code_execution_config=False,
    system_message="""
    SEO optimizer, you help the article writer suggest text modifications for search engine optimization. Ask the article writer to provide you with the text you need to optimize.
    When you respond with the status add the word TERMINATE.
    """,
    function_map={
        "answer_neuroscience_question": get_response_from_query
    }
)

def check_for_termination(message):
    # print("check_for_termination", message)
    content = message.get("content")
    if content != None and ("BRKT" in content or "TERMINATE" in content):
        print("should terminate")
        return True
    return False

user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=5,
    is_termination_msg=check_for_termination,
    code_execution_config=False,
    function_map={
        "answer_neuroscience_question": get_response_from_query
    }
)

In [35]:
groupchat = autogen.GroupChat(agents=[user_proxy, fact_checker, article_writer, seo_optimizer], messages=[], max_round=50)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [36]:
user_proxy.initiate_chat(
    manager,
    clear_history=True,
    message="""
    Write an article on the benefits of ashwagandha for people interested in nutrition, exercise, and a healthy lifestyle.
""",
)

user_proxy (to chat_manager):


    Write an article on the benefits of ashwagandha for people interested in nutrition, exercise, and a healthy lifestyle.


--------------------------------------------------------------------------------
article_writer (to chat_manager):

Title: Unveiling the Benefits of Ashwagandha: A Powerhouse for Nutrition, Exercise, and a Healthy Lifestyle

Introduction

In the realm of nutrition and wellness, Ashwagandha, a revered herb in Ayurvedic medicine, has been making waves. Known scientifically as Withania somnifera, Ashwagandha is often referred to as the "Indian ginseng" due to its rejuvenating properties. But what exactly are the benefits of this ancient herb, and how can it contribute to a healthy lifestyle, particularly for those interested in nutrition and exercise? Let's delve into the world of Ashwagandha and uncover its potential benefits.

The Power of Adaptogens

Ashwagandha belongs to a class of herbs known as "adaptogens," which are believed 